### Etymology

In [1]:
import pandas as pd
import numpy as np
import glob
import nltk
from nltk.stem import WordNetLemmatizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [2]:
lang_folder = "Turkish"

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Result/Word/Merge/Word_Merge2.csv")
df_word = df_word.head(10000)
df_word

In [ ]:
#word_lemma = []
#for word in df_word["word"]:
#    word_lemma.append(WordNetLemmatizer().lemmatize(word))
#    word_unique = set(word_lemma)

In [ ]:
#len(word_lemma)

In [ ]:
#len(word_unique)

In [ ]:
word_list = df_word["word"].to_list()

#### Nisanyan

In [94]:
response = requests.get(f"https://www.nisanyansozluk.com/kelime/armut")

In [106]:
soup = BeautifulSoup(response.content, 'html.parser')
#soup

In [96]:
#print(soup.prettify())

In [113]:
data = soup.find('div', attrs={"id":"__next"})
data

<div data-reactroot="" id="__next"><div><div class="sc-fe47d614-0 hWINya"></div></div></div>

In [116]:
url = "https://www.nisanyansozluk.com/kelime/armut"

In [117]:
html = urlopen(url)

In [118]:
data = BeautifulSoup(html, 'html.parser')

FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

#### Etimoloji Türkçe

In [4]:
response = requests.get(f"https://www.etimolojiturkce.com/kelime/moral")

In [5]:
soup = BeautifulSoup(response.content, 'lxml')
#soup

In [6]:
data = soup.find('div', attrs={"class":"holder"})
#data

In [7]:
data2 = data.find('div', attrs={"class":"main"})
#data2

In [8]:
data3 = data2.find('h1').get_text()
data3

'Moral'

In [9]:
data32 = data2.find_all('p')
data32[3].get_text()

' Fransızca\r morale "1. ahlakiyat, ahlak gücü, 2. ruh hali" sözcüğünden alıntıdır. Fransızca sözcük Fransızca\r moral "ahlaki, ahlaka ilişkin" sözcüğünün dişil (müennes, feminine) halidir. Bu sözcük Latince aynı anlama gelen moralis sözcüğünden alıntıdır. Latince sözcük Latince mos, mor- "mizaç, terbiye, ahlak" sözcüğünden +alis sonekiyle türetilmiştir. Latince sözcük Hintavrupa Anadilinde yazılı örneği bulunmayan *mō-s- biçiminden evrilmiştir. Bu biçim Hintavrupa Anadilinde yazılı örneği bulunmayan *mē-1 "ruhen eğilimi olmak" kökünden türetilmiştir. '

In [10]:
word_list = ["moral","takım,","elma","doktor"]

In [12]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://www.etimolojiturkce.com/kelime/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('div', attrs={"class":"holder"})
        word2 = word.find('div', attrs={"class":"main"})
        word3 = word2.find('h1').get_text()
        koken = word2.find_all('p')
        koken2 = koken[3].get_text()
                
        #iter_values = zip(word3, koken2)
        #for a, b in iter_values:
        #    var1 = a  # span
        #    var2 = b  # string
        #   
        #    result_list.append([i, var1, var2])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"word", 2:"koken"}, inplace=True)
df_result

,search_word,word,koken
0,moral,M,
1,moral,o,F
2,moral,r,r
3,moral,a,a
4,moral,l,n
5,elma,E,
6,elma,l,E
7,elma,m,s
8,elma,a,k
9,doktor,D,


In [ ]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://www.nisanyansozluk.com/kelime/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find('div', attrs={"class":"sc-7f314b79-0"})
        word = data.findAll('div', attrs={"class":"sc-7f314b79-2"})
        root = data.findAll('div', attrs={"class":"sc-7f314b79-12 fjUfSp"})
        
        iter_values = zip(word, root)
        for a, b in iter_values:
            var1 = a.span.get_text()  # span
            var2 = b.span.get_text()  # string
           
            result_list.append([i, var1, var2])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"dict_entry", 2:"language", 3:"definition", 4:"href"}, inplace=True)
df_result

In [ ]:
df_result.to_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_Result.xlsx", index=False)